In [2]:
import time

from utils import dataStructures, dataset, parallelizer
from utils.generators import markovian
import numpy as np


In [3]:
def print_correct(mela_id='22', janya_id='22_a', num_surr=100, surr_serial=False, time_cal=None):
    dat = dataset.GetRagaSongCoordsConcat(mela_id, janya_id)
    # dat1 = dataset.GetRagaSongCoords(janya_id)
    if time_cal is not None:
        assert isinstance(time_cal, dict)
    print("Starting surrogate generation...")
    t1 = time.perf_counter()
    d_surr1 = markovian.GenerateForRaga(mela_id, num_surr, serial=surr_serial)
    d_surr2 = markovian.GenerateForRaga(janya_id, num_surr, serial=surr_serial)
    t2 = time.perf_counter()
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    # for key in dat1:
    #     dat1[key] = dataStructures.PackTuples(*dat1[key])

    d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, d_surr2)
    adj = dataset.GetAdjustedMelodies(d_fin1, pre_min_len=35000)
    print("finding causality....")
    t3 = time.perf_counter()
    tc, totc, strengths = parallelizer.TrueLZPCausality_listParallel(list(adj.values()), list(adj.keys()), mela=dataset.GetRagaFromRagaId(mela_id))
    t4 = time.perf_counter()

    if time_cal is not None:
        time_cal["Surrogate{}".format(mela_id)] = t2 - t1
        time_cal["Causality{}".format(mela_id)] = t4 - t3
    return tc, totc, strengths

def get_generation_time50(mela_id='22', janya_id='22_a', surr_serial=True):
    dat = dataset.GetRagaSongCoordsConcat(mela_id)
    dat1 = dataset.GetRagaSongCoords(janya_id)
    print("Starting surrogate generation...")
    t1 = time.perf_counter()
    d_surr1 = markovian.GenerateForRaga(mela_id, 50, serial=surr_serial)
    d_surr2 = markovian.GenerateForRaga(janya_id, 50, serial=surr_serial)
    t2 = time.perf_counter()
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    for key in dat1:
        dat1[key] = dataStructures.PackTuples(*dat1[key])

    d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, dat1, d_surr2)
    adj = dataset.GetAdjustedMelodies(d_fin1)

    return t2 - t1


def ragaIterator(n_iters, mela, janya, num_surrogates=100, write=True, times_it=None):
    if times_it is not None:
        assert isinstance(times_it, dict)
        times_it["Surrogate{}".format(mela)] = []
        times_it["Causality{}".format(mela)] = []
    tcs = []
    strengths = []
    totcs = []
    f = open("/home/abhisheknandekar/surr_stats/{}/truecauses{}.txt".format(mela, num_surrogates), "a")
    t1 = time.perf_counter()
    for i in range(n_iters):
        print("iteration: ", i + 1)
        tm = {}
        if i % 2 != 0:
            tc, totc, st = print_correct(mela, janya, num_surr=num_surrogates, surr_serial=True, time_cal=tm)
        else:
            tc, totc, st = print_correct(mela, janya, num_surr=num_surrogates, surr_serial=True, time_cal=tm)
        if times_it is not None:
            times_it["Surrogate{}".format(mela)].append(tm["Surrogate{}".format(mela)])
            times_it["Causality{}".format(mela)].append(tm["Causality{}".format(mela)])
        if write:
            f.write("iteration: {}/{}, {}/{}; time taken: {}/{}\n".format(i+1, n_iters, tc, totc,
                                                                          tm["Surrogate{}".format(mela)],
                                                                          tm["Causality{}".format(mela)]))
        t = time.perf_counter()
        print("time elapsed: ", (t - t1))
        t1 = t
        tcs.append(tc)
        totcs.append(totc)
        strengths.append(st)
    f.close()
    return tcs, totcs, strengths

avg_conns = lambda t_c : np.array(t_c).mean()

## 100 Surrogates

In [1]:
t_dict = {}

In [4]:
tc15, totc15, ss15 = ragaIterator(10, '15', '15_m', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:28<00:00,  3.49it/s]


finding causality....
9136 true causes out of 11024 total connections
time elapsed:  659.1460676810001
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:34<00:00,  2.88it/s]


finding causality....
8604 true causes out of 11024 total connections
time elapsed:  662.554892328
iteration:  3
Starting surrogate generation...


...

finding causality....
9203 true causes out of 11024 total connections
time elapsed:  649.5606253370006
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:24<00:00,  4.15it/s]


finding causality....
8938 true causes out of 11024 total connections
time elapsed:  661.9708455669997
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:30<00:00,  3.26it/s]


finding causality....
9033 true causes out of 11024 total connections
time elapsed:  682.8012124899997


In [5]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

(8966.4, 62.99191391050008, 591.8526621072998)

In [6]:
tc8, totc8, ss8 = ragaIterator(10, '8', '8_d', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:44<00:00,  2.23it/s]


finding causality....
2345 true causes out of 11340 total connections
time elapsed:  857.3662240640006
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:31<00:00,  3.15it/s]


finding causality....
2355 true causes out of 11340 total connections
time elapsed:  831.4755920369989
iteration:  3
Starting surrogate generation...


...

finding causality....
2112 true causes out of 11340 total connections
time elapsed:  839.3787903020002
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:52<00:00,  1.90it/s]


finding causality....
2299 true causes out of 11340 total connections
time elapsed:  853.146232993
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:42<00:00,  2.36it/s]


finding causality....
2203 true causes out of 11340 total connections
time elapsed:  840.0987311130011


In [7]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

(2340.1, 71.91551117910012, 759.5560830447998)

In [8]:
tc22, totc22, ss22 = ragaIterator(10, '22', '22_a', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:31<00:00,  3.19it/s]


finding causality....
9141 true causes out of 10918 total connections
time elapsed:  704.242323855
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:33<00:00,  2.99it/s]


finding causality....
9660 true causes out of 10918 total connections
time elapsed:  703.9211227010019
iteration:  3
Starting surrogate generation...


...

finding causality....
8917 true causes out of 10918 total connections
time elapsed:  705.9565186299988
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:32<00:00,  3.11it/s]


finding causality....
9460 true causes out of 10918 total connections
time elapsed:  713.6176983299993
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:31<00:00,  3.22it/s]


finding causality....
9305 true causes out of 10918 total connections
time elapsed:  708.3508505570026


In [9]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

(9203.4, 68.32840259190053, 628.1603219855002)

In [10]:
tc28, totc28, ss28 = ragaIterator(10, '28', '28_k', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:35<00:00,  2.80it/s]


finding causality....
4587 true causes out of 10920 total connections
time elapsed:  742.0782401830002
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:28<00:00,  3.54it/s]


finding causality....
4586 true causes out of 10920 total connections
time elapsed:  723.3297505360024
iteration:  3
Starting surrogate generation...


...

finding causality....
3840 true causes out of 10920 total connections
time elapsed:  750.5826354720011
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:31<00:00,  3.20it/s]


finding causality....
3807 true causes out of 10920 total connections
time elapsed:  721.8894819339985
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


finding causality....
4661 true causes out of 10920 total connections
time elapsed:  722.3627313260004


In [11]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

(4339.5, 66.82670250039992, 652.9822392487004)

In [12]:
tc29, totc29, ss29 = ragaIterator(10, '29', '29_h', num_surrogates=100, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:41<00:00,  2.42it/s]


finding causality....
7723 true causes out of 11024 total connections
time elapsed:  756.4134862710016
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:31<00:00,  3.20it/s]


finding causality....
8017 true causes out of 11024 total connections
time elapsed:  742.8152989769987
iteration:  3
Starting surrogate generation...


...

finding causality....
8231 true causes out of 11024 total connections
time elapsed:  751.3106687530017
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:45<00:00,  2.22it/s]


finding causality....
7967 true causes out of 11024 total connections
time elapsed:  766.7958355509982
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:26<00:00,  3.72it/s]


finding causality....
7753 true causes out of 11024 total connections
time elapsed:  741.6658032949999


In [13]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

(7759.4, 66.99049650190027, 667.7164011065994)

In [14]:
t_dict


{'Surrogate15': [53.26868949300001,
  57.416984490000004,
  65.63219295199997,
  74.16984343900003,
  74.61059070900001,
  58.23887661700019,
  68.6738725109999,
  48.17700101700029,
  55.440457104000416,
  74.29063077299998],
 'Causality15': [592.3598863940001,
  591.381109908,
  592.825908996,
  587.7917414099998,
  595.133812782,
  587.7548258839997,
  594.1940579100001,
  588.1832207909993,
  593.7933637599999,
  595.1086932379994],
 'Surrogate8': [89.54098272700048,
  58.45198296499984,
  73.39187069900072,
  68.46543324499908,
  54.848082572998464,
  73.66247698000006,
  97.71908451700074,
  57.23644118000084,
  78.49121733200082,
  67.34753957300018],
 'Causality8': [755.0378918679999,
  759.8611159019993,
  758.2308762759985,
  752.4552812790007,
  760.3299102830006,
  759.4336225900006,
  759.0644153419998,
  769.2846034569993,
  761.8674483529994,
  759.9956650980002],
 'Surrogate22': [68.32396897400031,
  58.05357082699993,
  62.23361633500099,
  91.03038601200024,
  61.1071

## 50 Surrogates

In [3]:
t_dict = {}

In [ ]:
tc15, totc15, ss15 = ragaIterator(10, '15', '15_m', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


 92%|█████████▏| 46/50 [00:14<00:00,  6.13it/s]

In [ ]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

In [ ]:
tc8, totc8, ss8 = ragaIterator(10, '8', '8_d', num_surrogates=50, times_it=t_dict)

In [ ]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

In [ ]:
tc22, totc22, ss22 = ragaIterator(10, '22', '22_a', num_surrogates=50, times_it=t_dict)

In [ ]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

In [ ]:
tc28, totc28, ss28 = ragaIterator(10, '28', '28_k', num_surrogates=50, times_it=t_dict)

In [ ]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

In [ ]:
tc29, totc29, ss29 = ragaIterator(10, '29', '29_h', num_surrogates=50, times_it=t_dict)

In [ ]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

In [ ]:
t_dict

In [3]:
gentimes8 = []

for _ in range(10):
    gentimes8.append(get_generation_time50('8', '8_d'))
    gentimes8.append(get_generation_time50('15', '15_m'))
    gentimes8.append(get_generation_time50('22', '22_a'))
    gentimes8.append(get_generation_time50('28', '28_k'))
    gentimes8.append(get_generation_time50('29', '29_h'))

Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting s

In [5]:
gentime8 = np.array(gentimes8[0:10]).mean()
gentime15 = np.array(gentimes8[10:20]).mean()
gentime22 = np.array(gentimes8[20:30]).mean()
gentime28 = np.array(gentimes8[30:40]).mean()
gentime29 = np.array(gentimes8[40:50]).mean()

gentime8, gentime15, gentime22, gentime28, gentime29

(33.169615245600006,
 32.59101138219999,
 32.932949019099965,
 31.621566379899967,
 32.94254417129996)

## 150 surrogates

In [6]:
t_dict = {}

In [ ]:
tc15, totc15, ss15 = ragaIterator(10, '15', '15_m', num_surrogates=150, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 150/150 [00:54<00:00,  2.74it/s]


finding causality....
20336 true causes out of 24024 total connections
time elapsed:  1912.7792934960016
iteration:  2
Starting surrogate generation...


100%|██████████| 150/150 [00:35<00:00,  4.24it/s]


finding causality....
19595 true causes out of 24024 total connections
time elapsed:  1888.3809031659985
iteration:  3
Starting surrogate generation...


...

finding causality....
20604 true causes out of 24024 total connections
time elapsed:  1900.1705254010012
iteration:  5
Starting surrogate generation...


100%|██████████| 150/150 [00:51<00:00,  2.91it/s]


finding causality....
20164 true causes out of 24024 total connections
time elapsed:  1918.1216722840036
iteration:  6
Starting surrogate generation...


100%|██████████| 150/150 [00:45<00:00,  3.26it/s]


finding causality....


In [ ]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

In [12]:
tc8, totc8, ss8 = ragaIterator(10, '8', '8_d', num_surrogates=150, times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
14463 true causes out of 24648 total connections
time elapsed:  2377.1097622410016
iteration:  2
Starting surrogate generation...
finding causality....
15204 true causes out of 24648 total connections
time elapsed:  2329.3596639659954
iteration:  3
Starting surrogate generation...
finding causality....
15244 true causes out of 24648 total connections
time elapsed:  2377.415402065002
iteration:  4
Starting surrogate generation...
finding causality....


Process ForkPoolWorker-1040:
Process ForkPoolWorker-1047:
Process ForkPoolWorker-1052:
Process ForkPoolWorker-1055:
Process ForkPoolWorker-1034:
Process ForkPoolWorker-1054:
Process ForkPoolWorker-1051:
Process ForkPoolWorker-1045:
Process ForkPoolWorker-1046:
Process ForkPoolWorker-1049:
Process ForkPoolWorker-1039:
Process ForkPoolWorker-1041:
Process ForkPoolWorker-1056:
Process ForkPoolWorker-1035:
Process ForkPoolWorker-1033:
Process ForkPoolWorker-1043:
Process ForkPoolWorker-1048:
Process ForkPoolWorker-1053:
Process ForkPoolWorker-1038:
Process ForkPoolWorker-1050:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/abhisheknandekar/miniconda3/envs/raga/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/abhisheknandekar/miniconda3

KeyboardInterrupt: 

In [ ]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

In [ ]:
tc22, totc22, ss22 = ragaIterator(10, '22', '22_a', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

In [ ]:
tc28, totc28, ss28 = ragaIterator(10, '28', '28_k', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

In [ ]:
tc29, totc29, ss29 = ragaIterator(10, '29', '29_h', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

In [ ]:
t_dict